In [1]:
import pandas as pd


In [2]:
# Load the patient_id_month_year file
url = "https://docs.google.com/spreadsheets/d/1nry5xBNR45TsrHKt0u1wBj7LFtrS7aN9M2O_Jb5tSvw/export?format=csv"
df = pd.read_csv(url)

In [3]:
df

,patient_id,month_year
0,ID0001,1/1/2023
1,ID0001,2/1/2023
2,ID0001,3/1/2023
3,ID0001,6/1/2023
4,ID0001,8/1/2023
...,...,...
7304,ID1000,2/1/2023
7305,ID1000,5/1/2023
7306,ID1000,7/1/2023
7307,ID1000,10/1/2023


In [4]:
# Convert month_year to datetime
df['month_year'] = pd.to_datetime(df['month_year'])

In [5]:
# Sort by patient_id and month_year
df = df.sort_values(['patient_id', 'month_year'])

In [6]:
df

,patient_id,month_year
0,ID0001,2023-01-01
1,ID0001,2023-02-01
2,ID0001,2023-03-01
3,ID0001,2023-06-01
4,ID0001,2023-08-01
...,...,...
7304,ID1000,2023-02-01
7305,ID1000,2023-05-01
7306,ID1000,2023-07-01
7307,ID1000,2023-10-01


In [8]:
#Identify gaps between months (non-consecutive months start a new enrollment period)
df['month_diff'] = df.groupby('patient_id')['month_year'].diff().dt.days.fillna(0)

In [10]:
df

,patient_id,month_year,month_diff
0,ID0001,2023-01-01,0.0
1,ID0001,2023-02-01,31.0
2,ID0001,2023-03-01,28.0
3,ID0001,2023-06-01,92.0
4,ID0001,2023-08-01,61.0
...,...,...,...
7304,ID1000,2023-02-01,0.0
7305,ID1000,2023-05-01,89.0
7306,ID1000,2023-07-01,61.0
7307,ID1000,2023-10-01,92.0


In [11]:
df['new_enrollment'] = (df['month_diff'] > 31).cumsum()

In [12]:
df

,patient_id,month_year,month_diff,new_enrollment
0,ID0001,2023-01-01,0.0,0
1,ID0001,2023-02-01,31.0,0
2,ID0001,2023-03-01,28.0,0
3,ID0001,2023-06-01,92.0,1
4,ID0001,2023-08-01,61.0,2
...,...,...,...,...
7304,ID1000,2023-02-01,0.0,2102
7305,ID1000,2023-05-01,89.0,2103
7306,ID1000,2023-07-01,61.0,2104
7307,ID1000,2023-10-01,92.0,2105


In [25]:
# Aggregate to get enrollment_start_date and enrollment_end_date
enrollment = df.groupby(['patient_id', 'new_enrollment']).agg(
    enrollment_start_date=('month_year', 'min'),
    enrollment_end_date=('month_year', 'max')
).reset_index()

In [26]:
enrollment

,patient_id,new_enrollment,enrollment_start_date,enrollment_end_date
0,ID0001,0,2023-01-01,2023-03-01
1,ID0001,1,2023-06-01,2023-06-01
2,ID0001,2,2023-08-01,2023-08-01
3,ID0001,3,2023-11-01,2023-12-01
4,ID0002,3,2023-02-01,2023-04-01
...,...,...,...,...
3100,ID0999,2102,2023-12-01,2023-12-01
3101,ID1000,2102,2023-02-01,2023-02-01
3102,ID1000,2103,2023-05-01,2023-05-01
3103,ID1000,2104,2023-07-01,2023-07-01


In [27]:
# Save to CSV
enrollment.to_csv('patient_enrollment_span.csv', index=False)

In [28]:
print("Step 1 Answer: Number of rows in patient_enrollment_span.csv:", enrollment.shape[0])

Step 1 Answer: Number of rows in patient_enrollment_span.csv: 3105


In [29]:
# Step 2: Data Aggregation
# Load outpatient visits
url_visits = "https://docs.google.com/spreadsheets/d/1OVLFbEYaPlw0wqY01NBlLg9TneuImg6y0VLESvGQSNc/export?format=csv"
visits = pd.read_csv(url_visits)

In [30]:
visits

,patient_id,date,outpatient_visit_count
0,ID0001,1/3/2023,2
1,ID0001,1/8/2023,2
2,ID0001,1/9/2023,2
3,ID0001,1/15/2023,2
4,ID0001,1/21/2023,4
...,...,...,...
15323,ID0980,11/25/2023,2
15324,ID0980,10/20/2023,1
15325,ID0980,4/4/2023,1
15326,ID0980,7/26/2023,4


In [31]:
visits['date'] = pd.to_datetime(visits['date'])

In [32]:
visits

,patient_id,date,outpatient_visit_count
0,ID0001,2023-01-03,2
1,ID0001,2023-01-08,2
2,ID0001,2023-01-09,2
3,ID0001,2023-01-15,2
4,ID0001,2023-01-21,4
...,...,...,...
15323,ID0980,2023-11-25,2
15324,ID0980,2023-10-20,1
15325,ID0980,2023-04-04,1
15326,ID0980,2023-07-26,4


In [33]:
#Create an empty list to store results
result_rows = []

In [34]:
print(enrollment.columns)

Index(['patient_id', 'new_enrollment', 'enrollment_start_date',
       'enrollment_end_date'],
      dtype='object')


In [35]:
# Loop through each enrollment period
for _, row in enrollment.iterrows():
    pid = row['patient_id']
    start = row['enrollment_start_date']
    end = row['enrollment_end_date']
    
    patient_visits = visits[(visits['patient_id'] == pid) & 
                            (visits['date'] >= start) & 
                            (visits['date'] <= end)]
    
    ct_outpatient_visits = patient_visits['outpatient_visit_count'].sum()
    ct_days_with_outpatient_visit = patient_visits['date'].nunique()
    
    result_rows.append({
        'patient_id': pid,
        'enrollment_start_date': start,
        'enrollment_end_date': end,
        'ct_outpatient_visits': ct_outpatient_visits,
        'ct_days_with_outpatient_visit': ct_days_with_outpatient_visit
    })


In [36]:
result_df = pd.DataFrame(result_rows)

In [37]:
result_df

,patient_id,enrollment_start_date,enrollment_end_date,ct_outpatient_visits,ct_days_with_outpatient_visit
0,ID0001,2023-01-01,2023-03-01,13,6
1,ID0001,2023-06-01,2023-06-01,0,0
2,ID0001,2023-08-01,2023-08-01,0,0
3,ID0001,2023-11-01,2023-12-01,3,2
4,ID0002,2023-02-01,2023-04-01,7,2
...,...,...,...,...,...
3100,ID0999,2023-12-01,2023-12-01,0,0
3101,ID1000,2023-02-01,2023-02-01,0,0
3102,ID1000,2023-05-01,2023-05-01,0,0
3103,ID1000,2023-07-01,2023-07-01,0,0


In [38]:
# Save to CSV
result_df.to_csv('result.csv', index=False)

In [39]:
print("Step 2 Answer: Number of distinct values of ct_days_with_outpatient_visit:", 
      result_df['ct_days_with_outpatient_visit'].nunique())

Step 2 Answer: Number of distinct values of ct_days_with_outpatient_visit: 31
